### seq2seq 모델
- 추천 사이트: https://pasus.tistory.com/290

\begin{align} & \mathbf{x}_{raw} = [2, \ 8, \ 3, \ 9, \ 6] \\ \\ & \mathbf{x}'_{raw} = [0, \ 6, \ 9, \ 3, \ 8] \\ \\ & \mathbf{y}_{raw} = [6, \ 9, \ 3, \ 8, \ 2] \end{align}



\begin{align} \mathbf{x}_{raw} 는 인코더의 입력 시퀀스로 사용한다. \\ \mathbf{x}'_{raw} 는 디코더의 입력 시퀀스로 사용한다. \\ 맨 앞의 성분 
 0 은 'SOS' 를 표시한 것이다. \\ \mathbf{y}_{raw} 는 디코더가 생성해야 하는 참값으로 사용한다. \end{align}

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FlSVW8%2FbtsrsR3c2Qf%2FICKyYqu9IKk9afqvG3Q8IK%2Fimg.png">

return_state = True 를 한 경우에는 마지막 시퀀스에서의 출력(은닉상태), 은닉상태 와 셀상태(cell state) 값이 출력된다. 
가 인코더의 출력으로서 컨텍스트 벡터가 되며 이것이 
디코더의 초기 은닉상태와 셀상태로 전달된다.

1. 히든 상태 (Hidden State, h)
- 히든 상태는 LSTM이 지금까지 입력받은 정보의 요약이라고 할 수 있다.
- 이는 LSTM이 현재까지의 시퀀스를 보고 예측할 때 사용하는 정보.
- 히든 상태는 시퀀스의 각 타임스텝마다 업데이트되며, LSTM이 다음 타임스텝을 처리할 때 바로 이전 타임스텝의 히든 상태를 입력으로 사용한다.

구체적으로, 히든 상태는 다음과 같은 역할을 합니다:
-단기 메모리: 최근 시퀀스에서 중요한 정보를 유지합니다. 이 정보는 다음 타임스텝을 처리하거나 디코더에서 사용할 때 참고됩니다.
- 디코더에서 예측을 할 때, 인코더의 마지막 히든 상태는 디코더의 초기 히든 상태로 사용됩니다. 이로 인해 인코더가 학습한 중요한 정보가 디코더에 전달.

2. 셀 상태 (Cell State, c)
- 셀 상태는 LSTM이 장기적으로 기억하려는 정보를 저장.
- 셀 상태는 시퀀스 전체에서 정보가 점진적으로 업데이트되며, 시퀀스 내에서 중요한 정보를 기억하고 필요에 따라 정보를 제거하거나 추가.
- 셀 상태는 LSTM의 핵심으로, 장기 의존성 문제(Long-term dependency)를 해결하는 데 기여합니다. 이는 시퀀스의 앞쪽에 있는 중요한 정보를 잃지 않고 뒤쪽으로 잘 전달될 수 있도록 한다.

구체적으로, 셀 상태는 다음과 같은 역할을 합니다:
- 장기 메모리: 시퀀스 전반에서 지속적으로 정보를 유지하거나 제거합니다. 이 정보는 필요에 따라 적절하게 갱신되며, 중요한 패턴이나 정보를 장기적으로 저장.
- 인코더에서 마지막 셀 상태는 디코더의 초기 셀 상태로 사용되어, 시퀀스 전반에서 인코더가 학습한 중요한 정보가 디코더에 전달.

#### (히든 상태와 셀 상태의 차이점)
히든 상태는 현재 시점의 출력을 생성하는 데 직접적으로 사용됩니다. 이는 LSTM이 지금까지의 입력을 바탕으로 예측을 할 때 참고하는 정보입니다.
셀 상태는 장기적인 정보 저장소로, 시퀀스 전반에 걸쳐 중요한 정보를 유지하거나 필요 없어진 정보를 제거하는 데 사용됩니다.

#### 요약
- 히든 상태 (h): 현재까지의 시퀀스를 요약한 값으로, LSTM의 즉각적인 출력과 예측에 직접적인 영향을 미칩니다. 단기 메모리라고 할 수 있습니다.
- 셀 상태 (c): 시퀀스 전반에서 중요한 정보를 장기적으로 저장하고 필요 없을 때 제거합니다. 장기 메모리라고 할 수 있습니다.

In [ ]:
input                  output
-----------------------------------
[3, 5, 0, 0, 6] 	 [6, 0, 0, 5, 3] 	
[1, 5, 0, 8, 0] 	 [0, 8, 0, 5, 1] 	
[0, 6, 6, 1, 1] 	 [1, 1, 6, 6, 0] 	 
[1, 6, 6, 2, 8] 	 [8, 2, 6, 6, 1] 	 
[2, 1, 2, 6, 0] 	 [0, 6, 2, 1, 2] 	 
[2, 4, 8, 3, 7] 	 [7, 3, 8, 4, 2] 	 
[4, 7, 2, 3, 7] 	 [7, 3, 2, 7, 4] 	 
[5, 1, 8, 3, 6] 	 [6, 3, 8, 1, 5] 	
[4, 5, 9, 9, 7] 	 [7, 9, 9, 5, 4] 	 
[6, 3, 7, 7, 0] 	 [0, 7, 7, 3, 6] 	

In [ ]:
### 전체코드

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, Dense

# 데이터 준비
input_data = [
    [3, 5, 0, 0, 6],
    [1, 5, 0, 8, 0],
    [0, 6, 6, 1, 1],
    [1, 6, 6, 2, 8],
    [2, 1, 2, 6, 0],
    [2, 4, 8, 3, 7],
    [4, 7, 2, 3, 7],
    [5, 1, 8, 3, 6],
    [4, 5, 9, 9, 7],
    [6, 3, 7, 7, 0]
]

output_data = [
    [6, 0, 0, 5, 3],
    [0, 8, 0, 5, 1],
    [1, 1, 6, 6, 0],
    [8, 2, 6, 6, 1],
    [0, 6, 2, 1, 2],
    [7, 3, 8, 4, 2],
    [7, 3, 2, 7, 4],
    [6, 3, 8, 1, 5],
    [7, 9, 9, 5, 4],
    [0, 7, 7, 3, 6]
]

input_data = np.array(input_data)
output_data = np.array(output_data)

# 원-핫 인코딩
num_tokens = 10  # 각 값이 0~9이므로 토큰의 총 개수는 10개
input_data_one_hot = np.eye(num_tokens)[input_data]
output_data_one_hot = np.eye(num_tokens)[output_data]

# 인코더 정의
encoder_inputs = Input(shape=(None, num_tokens))  # (시퀀스 길이, 입력 특성 수)
encoder_rnn = SimpleRNN(32, return_state=True)
encoder_outputs, state_h = encoder_rnn(encoder_inputs)
encoder_states = [state_h]  # 인코더의 상태 (hidden state)

# 디코더 정의
decoder_inputs = Input(shape=(None, num_tokens))  # (시퀀스 길이, 출력 특성 수)
decoder_rnn = SimpleRNN(32, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_rnn(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 학습
model.fit([input_data_one_hot, output_data_one_hot], output_data_one_hot, epochs=500, batch_size=16)

# 예제 데이터를 원핫 인코딩
test_input = np.array([[3, 5, 0, 0, 6]])
test_input_one_hot = np.eye(num_tokens)[test_input]  # (1, 5, 10) 형상

# 모델 전체에 대해 바로 예측
decoder_input = np.zeros((1, 5, num_tokens))  # 디코더 입력은 첫 타임스텝에서 빈 벡터로 시작
predicted_output = model.predict([test_input_one_hot, decoder_input])

# 예측 결과의 원핫 인코딩된 벡터를 다시 인덱스로 변환
predicted_sequence = np.argmax(predicted_output, axis=-1)
print("Predicted sequence:", predicted_sequence[0])


In [57]:

test_input = np.array([[3, 5, 0, 0, 6]])
test_input_one_hot = np.eye(num_tokens)[test_input]  # (1, 5, 10) 형상

# 모델 전체에 대해 바로 예측
decoder_input = np.zeros((1, 5, num_tokens))  # 디코더 입력은 첫 타임스텝에서 빈 벡터로 시작
predicted_output = model.predict([test_input_one_hot, decoder_input])
# 예측 결과의 원핫 인코딩된 벡터를 다시 인덱스로 변환
predicted_sequence = np.argmax(predicted_output, axis=-1)
print("Predicted sequence:", predicted_sequence[0])


[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted sequence: [6 0 0 5 3]


In [32]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 준비
input_data = [
    [3, 5, 0, 0, 6],
    [1, 5, 0, 8, 0],
    [0, 6, 6, 1, 1],
    [1, 6, 6, 2, 8],
    [2, 1, 2, 6, 0],
    [2, 4, 8, 3, 7],
    [4, 7, 2, 3, 7],
    [5, 1, 8, 3, 6],
    [4, 5, 9, 9, 7],
    [6, 3, 7, 7, 0]
]

output_data = [
    [6, 0, 0, 5, 3],
    [0, 8, 0, 5, 1],
    [1, 1, 6, 6, 0],
    [8, 2, 6, 6, 1],
    [0, 6, 2, 1, 2],
    [7, 3, 8, 4, 2],
    [7, 3, 2, 7, 4],
    [6, 3, 8, 1, 5],
    [7, 9, 9, 5, 4],
    [0, 7, 7, 3, 6]
]

input_data = np.array(input_data)
output_data = np.array(output_data)

# 원-핫 인코딩
num_tokens = 10  # 각 값이 0~9이므로 토큰의 총 개수는 10개
input_data_one_hot = np.eye(num_tokens)[input_data]
output_data_one_hot = np.eye(num_tokens)[output_data]

# 인코더 정의
encoder_inputs = Input(shape=(None, num_tokens))  # (시퀀스 길이, 입력 특성 수)
encoder_lstm = LSTM(32, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]  # 인코더의 상태 (hidden state, cell state)

# 디코더 정의
decoder_inputs = Input(shape=(None, num_tokens))  # (시퀀스 길이, 출력 특성 수)
decoder_lstm = LSTM(32, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 학습
model.fit([input_data_one_hot, output_data_one_hot], output_data_one_hot, epochs=500, batch_size=16)

# 예제 데이터를 원핫 인코딩
test_input = np.array([[3, 5, 0, 0, 6]])
test_input_one_hot = np.eye(num_tokens)[test_input]  # (1, 5, 10) 형상

# 모델 전체에 대해 바로 예측
decoder_input = np.zeros((1, 5, num_tokens))  # 디코더 입력은 첫 타임스텝에서 빈 벡터로 시작
predicted_output = model.predict([test_input_one_hot, decoder_input])

# 예측 결과의 원핫 인코딩된 벡터를 다시 인덱스로 변환
predicted_sequence = np.argmax(predicted_output, axis=-1)
print("Predicted sequence:", predicted_sequence[0])


# # 예측을 위한 인코더 모델
# encoder_model = Model(encoder_inputs, encoder_states)

# # 디코더 모델 설정
# decoder_state_input_h = Input(shape=(32,))
# decoder_state_input_c = Input(shape=(32,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_outputs, state_h, state_c = decoder_lstm(
#     decoder_inputs, initial_state=decoder_states_inputs)
# decoder_states = [state_h, state_c]
# decoder_outputs = decoder_dense(decoder_outputs)

# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs] + decoder_states)

# # 시퀀스 예측 함수
# def decode_sequence(input_seq):
#     # 인코더를 통해 상태를 얻음
#     states_value = encoder_model.predict(input_seq)

#     # 첫 디코더 입력은 모두 0인 벡터로 설정
#     target_seq = np.zeros((1, 1, num_tokens))

#     # 샘플링 루프
#     decoded_sequence = []
#     for _ in range(5):  # 최대 시퀀스 길이만큼 반복
#         output_tokens, h, c = decoder_model.predict(
#             [target_seq] + states_value)

#         # 결과를 샘플링
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         decoded_sequence.append(sampled_token_index)

#         # 다음 타겟 시퀀스 업데이트
#         target_seq = np.zeros((1, 1, num_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.

#         # 상태 업데이트
#         states_value = [h, c]

#     return decoded_sequence

# # 예측 예제
# test_input = np.array([[3, 5, 0, 0, 6]])
# test_input_one_hot = np.eye(num_tokens)[test_input]
# predicted_sequence = decode_sequence(test_input_one_hot)
# print("Predicted sequence:", predicted_sequence)


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0200 - loss: 2.3194
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0200 - loss: 2.3138
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0200 - loss: 2.3083
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0400 - loss: 2.3029
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0600 - loss: 2.2974
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1000 - loss: 2.2920
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1000 - loss: 2.2867
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1400 - loss: 2.2813
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1800 - loss: 2.2760
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1800 - loss: 2.2707
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.2000 - loss: 2.2653
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2000 - loss

In [41]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 준비
input_data = [
    [3, 5, 0, 0, 6],
    [1, 5, 0, 8, 0],
    [0, 6, 6, 1, 1],
    [1, 6, 6, 2, 8],
    [2, 1, 2, 6, 0],
    [2, 4, 8, 3, 7],
    [4, 7, 2, 3, 7],
    [5, 1, 8, 3, 6],
    [4, 5, 9, 9, 7],
    [6, 3, 7, 7, 0]
]

output_data = [
    [6, 0, 0, 5, 3],
    [0, 8, 0, 5, 1],
    [1, 1, 6, 6, 0],
    [8, 2, 6, 6, 1],
    [0, 6, 2, 1, 2],
    [7, 3, 8, 4, 2],
    [7, 3, 2, 7, 4],
    [6, 3, 8, 1, 5],
    [7, 9, 9, 5, 4],
    [0, 7, 7, 3, 6]
]

input_data = np.array(input_data)
output_data = np.array(output_data)

# 원-핫 인코딩
num_tokens = 10  # 각 값이 0~9이므로 토큰의 총 개수는 10개
input_data_one_hot = np.eye(num_tokens)[input_data]
output_data_one_hot = np.eye(num_tokens)[output_data]


In [42]:
input_data.shape

(10, 5)

In [43]:
input_data_one_hot.shape

(10, 5, 10)

In [ ]:
input_data_one_hot

In [ ]:
output_data_one_hot

In [46]:
num_tokens

10

In [47]:
# 인코더 정의
encoder_inputs = Input(shape=(None, num_tokens))  # (시퀀스 길이, 입력 특성 수)
encoder_lstm = LSTM(32, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]  # 인코더의 상태 (hidden state, cell state)

# 디코더 정의
decoder_inputs = Input(shape=(None, num_tokens))  # (시퀀스 길이, 출력 특성 수)
decoder_lstm = LSTM(32, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Seq2Seq 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 학습
model.fit([input_data_one_hot, output_data_one_hot], output_data_one_hot, epochs=500, batch_size=16)


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1400 - loss: 2.2792
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1800 - loss: 2.2738
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.2400 - loss: 2.2684
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2600 - loss: 2.2630
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2600 - loss: 2.2575
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2800 - loss: 2.2520
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.2800 - loss: 2.2465
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2800 - loss: 2.2409
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2800 - loss: 2.2352
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3000 - loss: 2.2295
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3400 - loss: 2.2238
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3400 - loss

In [48]:
# 예제 데이터를 원핫 인코딩
test_input = np.array([[3, 5, 0, 0, 6]])
test_input_one_hot = np.eye(num_tokens)[test_input]  # (1, 5, 10) 형상

# 모델 전체에 대해 바로 예측
decoder_input = np.zeros((1, 5, num_tokens))  # 디코더 입력은 첫 타임스텝에서 빈 벡터로 시작
predicted_output = model.predict([test_input_one_hot, decoder_input])

# 예측 결과의 원핫 인코딩된 벡터를 다시 인덱스로 변환
predicted_sequence = np.argmax(predicted_output, axis=-1)
print("Predicted sequence:", predicted_sequence[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Predicted sequence: [6 0 0 5 3]


In [53]:
# 예측을 위한 인코더 모델
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 모델 설정
decoder_state_input_h = Input(shape=(32,))
decoder_state_input_c = Input(shape=(32,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 시퀀스 예측 함수
def decode_sequence(input_seq):
    # 인코더를 통해 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # 첫 디코더 입력은 모두 0인 벡터로 설정
    target_seq = np.zeros((1, 1, num_tokens))

    # 샘플링 루프
    decoded_sequence = []
    for _ in range(5):  # 최대 시퀀스 길이만큼 반복
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 결과를 샘플링
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sequence.append(sampled_token_index)

        # 다음 타겟 시퀀스 업데이트
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태 업데이트
        states_value = [h, c]

    return decoded_sequence

# 예측 예제
test_input = np.array([[3, 5, 0, 0, 6]])
test_input_one_hot = np.eye(num_tokens)[test_input]
predicted_sequence = decode_sequence(test_input_one_hot)
print("Predicted sequence:", predicted_sequence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted sequence: [6, 0, 0, 5, 3]


In [50]:
# 예제 데이터를 원핫 인코딩
# test_input = np.array([[3, 5, 0, 0, 6]])
test_input = np.array([[0, 6, 6, 1, 1]])
test_input_one_hot = np.eye(num_tokens)[test_input]  # (1, 5, 10) 형상

# 모델 전체에 대해 바로 예측
decoder_input = np.zeros((1, 5, num_tokens))  # 디코더 입력은 첫 타임스텝에서 빈 벡터로 시작
predicted_output = model.predict([test_input_one_hot, decoder_input])

# 예측 결과의 원핫 인코딩된 벡터를 다시 인덱스로 변환
predicted_sequence = np.argmax(predicted_output, axis=-1)
print("Predicted sequence:", predicted_sequence[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted sequence: [0 8 2 1 2]


In [ ]:
input_data = [
    [3, 5, 0, 0, 6],
    [1, 5, 0, 8, 0],
    [0, 6, 6, 1, 1],
    [1, 6, 6, 2, 8],
    [2, 1, 2, 6, 0],
    [2, 4, 8, 3, 7],
    [4, 7, 2, 3, 7],
    [5, 1, 8, 3, 6],
    [4, 5, 9, 9, 7],
    [6, 3, 7, 7, 0]
]

output_data = [
    [6, 0, 0, 5, 3],
    [0, 8, 0, 5, 1],
    [1, 1, 6, 6, 0],
    [8, 2, 6, 6, 1],
    [0, 6, 2, 1, 2],
    [7, 3, 8, 4, 2],
    [7, 3, 2, 7, 4],
    [6, 3, 8, 1, 5],
    [7, 9, 9, 5, 4],
    [0, 7, 7, 3, 6]
]


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 입력 데이터
input_data = [
    "나는 학교에 간다",
    "나는 밥을 먹는다",
    "그는 운동을 한다",
    "우리는 공부를 한다",
    "그녀는 노래를 부른다"
]

# 출력 데이터
output_data = [
    "i go to school",
    "i eat food",
    "he exercises",
    "we study",
    "she sings"
]

# 토크나이저 정의 (한글)
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_data)
input_sequences = input_tokenizer.texts_to_sequences(input_data)

# 토크나이저 정의 (영어)
output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts(output_data)
output_sequences = output_tokenizer.texts_to_sequences(output_data)

# 패딩 처리 (시퀀스 길이를 동일하게 맞춤)
max_input_length = max(len(seq) for seq in input_sequences)
max_output_length = max(len(seq) for seq in output_sequences)

input_sequences_pad = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
output_sequences_pad = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')


In [9]:
input_tokenizer.index_word

{1: '나는',
 2: '한다',
 3: '학교에',
 4: '간다',
 5: '밥을',
 6: '먹는다',
 7: '그는',
 8: '운동을',
 9: '우리는',
 10: '공부를',
 11: '그녀는',
 12: '노래를',
 13: '부른다'}

In [10]:
output_tokenizer.index_word

{1: 'i',
 2: 'go',
 3: 'to',
 4: 'school',
 5: 'eat',
 6: 'food',
 7: 'he',
 8: 'exercises',
 9: 'we',
 10: 'study',
 11: 'she',
 12: 'sings'}

In [11]:
input_sequences


[[1, 3, 4], [1, 5, 6], [7, 8, 2], [9, 10, 2], [11, 12, 13]]

In [12]:
input_sequences_pad

array([[ 1,  3,  4],
       [ 1,  5,  6],
       [ 7,  8,  2],
       [ 9, 10,  2],
       [11, 12, 13]])

In [13]:
output_sequences

[[1, 2, 3, 4], [1, 5, 6], [7, 8], [9, 10], [11, 12]]

In [14]:
output_sequences_pad

array([[ 1,  2,  3,  4],
       [ 1,  5,  6,  0],
       [ 7,  8,  0,  0],
       [ 9, 10,  0,  0],
       [11, 12,  0,  0]])

In [15]:
import numpy as np

num_input_tokens = len(input_tokenizer.word_index) + 1
num_output_tokens = len(output_tokenizer.word_index) + 1

input_data_one_hot = np.eye(num_input_tokens)[input_sequences_pad]
output_data_one_hot = np.eye(num_output_tokens)[output_sequences_pad]


In [16]:
input_data_one_hot

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]])

In [17]:
num_input_tokens

14

In [ ]:
from tensorflow.keras.layers import Embedding

# 임베딩 설정
embedding_dim = 50  # 임베딩 벡터의 차원
# 인코더 임베딩 레이어
encoder_embedding = Embedding(input_dim=num_input_tokens, output_dim=embedding_dim,
                               input_length=max_input_length)(encoder_inputs)

# 디코더 임베딩 레이어
decoder_embedding = Embedding(input_dim=num_output_tokens, output_dim=embedding_dim, 
                              input_length=max_output_length)(decoder_inputs)


### 원핫 대신 임베딩의 사용
##### 임베딩 차원의 선택
임베딩 차원은 다음 요소들을 고려하여 설정합니다:

- 데이터 크기: 어휘(단어) 크기가 클수록 임베딩 차원을 더 크게 설정할 수 있습니다.
- 모델의 복잡도: 임베딩 차원이 클수록 모델의 파라미터 수가 증가합니다.
- 메모리 및 연산 비용: 차원이 너무 크면 메모리 사용량과 연산 비용이 크게 증가합니다.

#### +1을 하는 이유
- num_input_tokens와 num_output_tokens에서 +1을 하는 이유는 패딩(Padding) 또는 OOV(Out-Of-Vocabulary) 토큰을 처리하기 위해입니다.
- 인덱스 0 예약: Tokenizer를 사용하여 시퀀스를 숫자로 변환할 때, 일반적으로 인덱스 0은 패딩 토큰에 할당됩니다. 따라서 실제 단어 인덱스는 1부터 시작합니다. 그러므로 전체 단어 수를 고려할 때 인덱스를 하나 더 추가하여 +1을 합니다.
- 패딩 또는 OOV 처리: 인덱스 0은 패딩이나 OOV(Out-Of-Vocabulary) 토큰에 할당될 수 있기 때문에, 실제 어휘 크기보다 하나 더 큰 크기를 고려해야 합니다.

In [20]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 입력 데이터
input_data = [
    "나는 학교에 간다",
    "나는 밥을 먹는다",
    "그는 운동을 한다",
    "우리는 공부를 한다",
    "그녀는 노래를 부른다"
]

# 출력 데이터
output_data = [
    "i go to school",
    "i eat food",
    "he exercises",
    "we study",
    "she sings"
]

# 토크나이저 정의 (한글)
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_data)
input_sequences = input_tokenizer.texts_to_sequences(input_data)

# 토크나이저 정의 (영어)
output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts(output_data)
output_sequences = output_tokenizer.texts_to_sequences(output_data)

# 패딩 처리 (시퀀스 길이를 동일하게 맞춤)
max_input_length = max(len(seq) for seq in input_sequences)
max_output_length = max(len(seq) for seq in output_sequences)

input_sequences_pad = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
output_sequences_pad = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# 임베딩 설정
embedding_dim = 50  # 임베딩 벡터의 차원
num_input_tokens = len(input_tokenizer.word_index) + 1
num_output_tokens = len(output_tokenizer.word_index) + 1
print( num_input_tokens)
print( num_output_tokens)

# # 인코더 정의
# encoder_embedding = Embedding(input_dim=num_input_tokens, output_dim=embedding_dim)(encoder_inputs)
# # 디코더 정의
# decoder_embedding = Embedding(input_dim=num_output_tokens, output_dim=embedding_dim)(decoder_inputs)



14
13


In [2]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 입력 데이터
input_data = [
    "나는 학교에 간다",
    "나는 밥을 먹는다",
    "그는 운동을 한다",
    "우리는 공부를 한다",
    "그녀는 노래를 부른다"
]

# 출력 데이터
output_data = [
    "i go to school",
    "i eat food",
    "he exercises",
    "we study",
    "she sings"
]

# 토크나이저 정의 (한글)
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_data)
input_sequences = input_tokenizer.texts_to_sequences(input_data)

# 토크나이저 정의 (영어)
output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts(output_data)
output_sequences = output_tokenizer.texts_to_sequences(output_data)

# 패딩 처리 (시퀀스 길이를 동일하게 맞춤)
max_input_length = max(len(seq) for seq in input_sequences)
max_output_length = max(len(seq) for seq in output_sequences)

input_sequences_pad = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
output_sequences_pad = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# 임베딩 설정
embedding_dim = 50  # 임베딩 벡터의 차원
num_input_tokens = len(input_tokenizer.word_index) + 1
num_output_tokens = len(output_tokenizer.word_index) + 1

# 인코더 정의
encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_dim=num_input_tokens, output_dim=embedding_dim)(encoder_inputs)
encoder_rnn = SimpleRNN(32, return_state=True)
encoder_outputs, state_h = encoder_rnn(encoder_embedding)

# 디코더 정의
decoder_inputs = Input(shape=(max_output_length,))
decoder_embedding = Embedding(input_dim=num_output_tokens, output_dim=embedding_dim)(decoder_inputs)
decoder_rnn = SimpleRNN(32, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_rnn(decoder_embedding, initial_state=[state_h])
decoder_dense = Dense(num_output_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 출력 데이터를 모델 학습에 맞게 변환 (sparse_categorical_crossentropy를 사용하기 때문에)
output_sequences_pad = np.expand_dims(output_sequences_pad, -1)

# 모델 학습
model.fit([input_sequences_pad, output_sequences_pad], output_sequences_pad, epochs=500, batch_size=16)

# 예측 예제
def predict_sequence(input_text):
    input_sequence = input_tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_input_length, padding='post')
    
    # 인코더 예측
    states_value = encoder_rnn.predict(input_sequence)

    # 디코더 입력
    target_sequence = np.zeros((1, 1))  # 첫 타임스텝은 시작 토큰
    predicted_sentence = []

    for _ in range(max_output_length):
        output_tokens, state_h = decoder_rnn.predict([target_sequence] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = output_tokenizer.index_word.get(sampled_token_index, '')
        predicted_sentence.append(sampled_word)

        # 종료 조건
        if sampled_word == '':
            break

        # 다음 입력으로 현재 예측된 단어 사용
        target_sequence = np.array([[sampled_token_index]])
        states_value = [state_h]

    return ' '.join(predicted_sentence)

# 테스트 예제
test_input = "나는 학교에 간다"
predicted_output = predict_sequence(test_input)
print("Input:", test_input)
print("Predicted Output:", predicted_output)


Epoch 1/500


c:\Python310\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_9', 'keras_tensor_13']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0500 - loss: 2.5680
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0500 - loss: 2.5234
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2500 - loss: 2.4789
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4000 - loss: 2.4343
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5000 - loss: 2.3891
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5500 - loss: 2.3432
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6000 - loss: 2.2965
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6500 - loss: 2.2489
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7500 - loss: 2.2003
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7500 - loss: 2.1507
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7500 - loss: 2.1002
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7500 - loss: 2.0489
Epo

AttributeError: 'SimpleRNN' object has no attribute 'predict'

In [9]:
# 입력 데이터
test_input = "나는 학교에 간다"
test_input_sequence = input_tokenizer.texts_to_sequences([test_input])
test_input_sequence = pad_sequences(test_input_sequence, maxlen=max_input_length, padding='post')

# 디코더 입력은 모두 0으로 시작
decoder_input = np.zeros((1, max_output_length))

# 전체 예측 수행
predicted_output = model.predict([test_input_sequence, decoder_input])

# 예측 결과를 단어로 변환
predicted_sequence = np.argmax(predicted_output, axis=-1)
predicted_sentence = ' '.join([output_tokenizer.index_word.get(idx, '') for idx in predicted_sequence[0]])

print("Predicted Output:", predicted_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Output: i   


In [10]:
def predict_sequence(input_text):
    # 입력 문장을 시퀀스로 변환하고 패딩
    input_sequence = input_tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_input_length, padding='post')

    # 인코더 상태 예측
    states_value = encoder_model.predict(input_sequence)

    # 첫 번째 디코더 입력은 시작 토큰(모두 0으로 시작)
    target_seq = np.zeros((1, 1))

    # 예측 결과 저장
    decoded_sentence = []

    for _ in range(max_output_length):
        # 다음 단어 예측
        output_tokens, states_value = decoder_model.predict([target_seq] + [states_value])

        # 예측된 단어 인덱스를 선택하고 저장
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = output_tokenizer.index_word.get(sampled_token_index, '')

        # 예측된 단어를 결과에 추가
        if sampled_word:
            decoded_sentence.append(sampled_word)

        # 종료 조건: 빈 단어 또는 최대 길이 도달
        if not sampled_word or len(decoded_sentence) >= max_output_length:
            break

        # 다음 입력으로 현재 예측된 단어를 설정
        target_seq = np.array([[sampled_token_index]])

    return ' '.join(decoded_sentence)

# 테스트 예제
test_input = "나는 학교에 간다"
predicted_output = predict_sequence(test_input)
print("Input:", test_input)
print("Predicted Output:", predicted_output)


NameError: name 'encoder_model' is not defined

In [12]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 입력 데이터
input_data = [
    "나는 학교에 간다",
    "나는 밥을 먹는다",
    "그는 운동을 한다",
    "우리는 공부를 한다",
    "그녀는 노래를 부른다"
]

# 출력 데이터
output_data = [
    "i go to school",
    "i eat food",
    "he exercises",
    "we study",
    "she sings"
]

# 토크나이저 정의 (한글)
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_data)
input_sequences = input_tokenizer.texts_to_sequences(input_data)

# 토크나이저 정의 (영어)
output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts(output_data)
output_sequences = output_tokenizer.texts_to_sequences(output_data)

# 패딩 처리 (시퀀스 길이를 동일하게 맞춤)
max_input_length = max(len(seq) for seq in input_sequences)
max_output_length = max(len(seq) for seq in output_sequences)

input_sequences_pad = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
output_sequences_pad = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# 임베딩 설정
embedding_dim = 50  # 임베딩 벡터의 차원
num_input_tokens = len(input_tokenizer.word_index) + 1
num_output_tokens = len(output_tokenizer.word_index) + 1

# 인코더 정의
encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_dim=num_input_tokens, output_dim=embedding_dim)(encoder_inputs)
encoder_rnn = SimpleRNN(32, return_state=True)
encoder_outputs, state_h = encoder_rnn(encoder_embedding)

# 디코더 정의
decoder_inputs = Input(shape=(max_output_length,))
decoder_embedding = Embedding(input_dim=num_output_tokens, output_dim=embedding_dim)(decoder_inputs)
decoder_rnn = SimpleRNN(32, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_rnn(decoder_embedding, initial_state=[state_h])
decoder_dense = Dense(num_output_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 출력 데이터를 모델 학습에 맞게 변환 (sparse_categorical_crossentropy를 사용하기 때문에)
output_sequences_pad = np.expand_dims(output_sequences_pad, -1)

# 모델 학습
model.fit([input_sequences_pad, output_sequences_pad], output_sequences_pad, epochs=500, batch_size=16)

# 인코더 모델 정의 (예측 단계에서 사용)
encoder_model = Model(encoder_inputs, state_h)

# 디코더 모델 정의 (예측 단계에서 사용)
decoder_state_input_h = Input(shape=(32,))
decoder_inputs_single = Input(shape=(1,))  # 예측 단계에서 단일 입력을 받기 위해 새로운 Input 정의

# 임베딩 레이어를 예측 단계에서 다시 정의
decoder_embedding_inf = Embedding(input_dim=num_output_tokens, output_dim=embedding_dim)(decoder_inputs_single)
decoder_outputs_inf, state_h_inf = decoder_rnn(decoder_embedding_inf, initial_state=[decoder_state_input_h])
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model = Model([decoder_inputs_single, decoder_state_input_h], [decoder_outputs_inf, state_h_inf])

# 예측 함수 정의
def predict_sequence(input_text):
    # 입력 문장을 시퀀스로 변환하고 패딩
    input_sequence = input_tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_input_length, padding='post')

    # 인코더 상태 예측
    states_value = encoder_model.predict(input_sequence)

    # 첫 번째 디코더 입력은 시작 토큰(모두 0으로 시작)
    target_seq = np.zeros((1, 1))

    # 예측 결과 저장
    decoded_sentence = []

    for _ in range(max_output_length):
        # 다음 단어 예측
        output_tokens, states_value = decoder_model.predict([target_seq, states_value])

        # 예측된 단어 인덱스를 선택하고 저장
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = output_tokenizer.index_word.get(sampled_token_index, '')

        # 예측된 단어를 결과에 추가
        if sampled_word:
            decoded_sentence.append(sampled_word)

        # 종료 조건: 빈 단어 또는 최대 길이 도달
        if not sampled_word or len(decoded_sentence) >= max_output_length:
            break

        # 다음 입력으로 현재 예측된 단어를 설정
        target_seq = np.array([[sampled_token_index]])

    return ' '.join(decoded_sentence)

# 테스트 예제
test_input = "나는 학교에 간다"
predicted_output = predict_sequence(test_input)
print("Input:", test_input)
print("Predicted Output:", predicted_output)


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0500 - loss: 2.5795
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0500 - loss: 2.5294
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2500 - loss: 2.4798
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4000 - loss: 2.4303
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4500 - loss: 2.3807
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5500 - loss: 2.3308
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6000 - loss: 2.2804
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6000 - loss: 2.2294
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6000 - loss: 2.1776
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6000 - loss: 2.1252
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5500 - loss: 2.0722
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6000 - loss

In [13]:
# 예측을 위한 간단한 코드
def simple_predict(input_text):
    # 입력 시퀀스를 토크나이저로 변환 및 패딩
    input_sequence = input_tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_input_length, padding='post')

    # 디코더 입력 (모두 0으로 시작)
    decoder_input = np.zeros((1, max_output_length))

    # 전체 모델을 이용해 예측
    predicted_output = model.predict([input_sequence, decoder_input])

    # 예측된 시퀀스를 단어로 변환
    predicted_sequence = np.argmax(predicted_output, axis=-1)
    predicted_sentence = ' '.join([output_tokenizer.index_word.get(idx, '') for idx in predicted_sequence[0]])

    return predicted_sentence

# 테스트 예제
test_input = "나는 학교에 간다"
predicted_output = simple_predict(test_input)
print("Input:", test_input)
print("Predicted Output:", predicted_output)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
Input: 나는 학교에 간다
Predicted Output: i   
